In [1]:
!git clone https://github.com/myprivatelearningacc/Autoscaling-Analysis.git

Cloning into 'Autoscaling-Analysis'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 68 (delta 23), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (68/68), 162.48 KiB | 7.06 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [2]:
%cd Autoscaling-Analysis

/content/Autoscaling-Analysis


In [3]:
!pip install gdown

In [4]:
!mkdir -p data/raw

In [5]:
!gdown 1-c20gJwbzjLsUWFFKbC1OOKxUb17MHSk -O data/raw/train.txt

Downloading...
From (original): https://drive.google.com/uc?id=1-c20gJwbzjLsUWFFKbC1OOKxUb17MHSk
From (redirected): https://drive.google.com/uc?id=1-c20gJwbzjLsUWFFKbC1OOKxUb17MHSk&confirm=t&uuid=c9be6e9d-f117-4cd8-a187-58c12bc0eb64
To: /content/Autoscaling-Analysis/data/raw/train.txt
100% 320M/320M [00:05<00:00, 63.2MB/s]


In [6]:
!gdown 1PaS5kJruNOy8OY6BZD_Ze5r3Y0Gowfse -O data/raw/test.txt

Downloading...
From: https://drive.google.com/uc?id=1PaS5kJruNOy8OY6BZD_Ze5r3Y0Gowfse
To: /content/Autoscaling-Analysis/data/raw/test.txt
100% 56.8M/56.8M [00:00<00:00, 60.4MB/s]


In [7]:
from pathlib import Path

TRAIN_PATH = Path("data/raw/train.txt")
TEST_PATH = Path("data/raw/test.txt")
print("File exists:", TRAIN_PATH.exists())
print("File size (MB):", TRAIN_PATH.stat().st_size / 1e6)

print("File exists:", TEST_PATH.exists())
print("File size (MB):", TEST_PATH.stat().st_size / 1e6)


File exists: True
File size (MB): 319.688542
File exists: True
File size (MB): 56.829318


In [8]:
!ls src

feature.py  forecasting.py  __init__.py  preprocessing.py


In [9]:
import pandas as pd

from src.preprocessing import load_and_clean_log
from src.forecasting import (
    train_arima,
    forecast_arima,
    train_xgboost,
    forecast_xgboost,
    evaluate_forecast
)
from src.feature import make_features

In [10]:
TRAIN_PATH = Path("data/raw/train.txt")
df_train = load_and_clean_log(TRAIN_PATH)

TEST_PATH = Path("data/raw/test.txt")
df_test = load_and_clean_log(TEST_PATH)

In [11]:
df_train = make_features(df_train)
df_test = make_features(df_test)
X_train = df_train.drop(columns=["requests"])
y_train = df_train["requests"]

X_test = df_test.drop(columns=["requests"])
y_test = df_test["requests"]

In [12]:
arima_model = train_arima(y_train, order=(2,1,2))
y_pred_arima = forecast_arima(arima_model, steps=len(y_test))

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/p

In [13]:
arima_metrics = evaluate_forecast(y_test.values, y_pred_arima.values)
arima_metrics

{'RMSE': np.float64(141.6844712775668),
 'MAE': 102.56079340054916,
 'MAPE': np.float64(58.94642081098759)}

In [14]:
#XGBoost
xgb_model = train_xgboost(X_train, y_train)
y_pred_xgb = forecast_xgboost(xgb_model, X_test)
xgb_metrics = evaluate_forecast(y_test.values, y_pred_xgb)
xgb_metrics

{'RMSE': np.float64(17.28441161884794),
 'MAE': 12.769179344177246,
 'MAPE': np.float64(10.159670717329545)}

In [15]:
pd.DataFrame.from_dict(
    {
        "ARIMA": arima_metrics,
        "XGBoost": xgb_metrics
    },
    orient="index"
)


,RMSE,MAE,MAPE
ARIMA,141.684471,102.560793,58.946421
XGBoost,17.284412,12.769179,10.159671
